In [22]:
import pandas as pd
import shutil
import os
import pydicom as pydicom
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [23]:
#!nvidia-smi

In [24]:
############################################# 
#.........FUNCIONES PREPROCESADO.............
#############################################

#FUNCIONES NECESARIAS PARA DICOM / PREPROCESADO DE LA IMAGEN (SON 5 PASOS):
#1. CARGAR LA IMAGEN:
def load_scan(path): #will load all DICOM images from a folder into a list for manipulation.
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices

#2. CONVERTIR EL RAW DATA EN UNIDADES HOUNSFIELD, PASO PREVIO A NORMALIZAR. 
def get_pixels_hu(slices): #The voxel values in the images are raw. get_pixels_hu converts raw values into HU. The transformation is linear. Therefore, so long as you have a slope and an intercept, you can rescale a voxel value to HU.
                           #Both the rescale intercept and rescale slope are stored in the DICOM header at the time of image acquisition (these values are scanner-dependent, so you will need external information).
    
    image = np.stack([s.pixel_array for s in slices])
    hist(np.array(image))
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    
    
    # Set outside-of-scan pixels to 0. 
    # The intercept is usually -1024, so air is approximately 0 (sin unidad, es -1000 en UH).
    image[image == -2000] = 0
        # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

#3. NORMALIZAR:
def normalize(image):
    MIN_BOUND=15
    MAX_BOUND=100
    
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    
    image[image>1] = 0. #quitar información
    image[image<0] = 0.
    return image



#4. RESAMPLING (para que todas tengas la misma "resolución"): 
def resample(image, scan, new_spacing=[5,5,5]):
    # Determine current pixel spacing
    pi=scan[0].PixelSpacing
    pi=pi[0]
    spacing = np.array(scan[0].SliceThickness + pi,np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    import scipy
    from  scipy import ndimage
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing


#5. "PREPROCESADO" o RESAMPLING_AVANZADO (es decir, que todos los pacientes tengan el mismo número de slices): 
def preprocesado(pixels, slices, dir_final): #todos los pacientes tienen que tener el mismo númro de cortes. 
    #HAY QUE HACER UN RESIZE!!!!
    
    pix_resampled, spacing = resample(pixels, slices, [5,5,5])
    
    pixels_n=[]
    for s in pix_resampled:
        #pixels1=s.pixel_array
        pixels1=s
        pixels1=normalize(pixels1) #SE NORMALIZAN AQUÍ
        pixels1=list(pixels1)
        pixels_n.append(pixels1)
    pixels_n=np.array(pixels_n)
    
    '''n,m,l=pixels_n.shape    
    pixels_n=list(pixels_n)
    
    
    matriz_float = np.zeros((m, l))
    matriz_float = np.int16(matriz_float)
    for ind in range(n,41):
        pixels_n.append(matriz_float)
    pixels_n=np.array(pixels_n)'''
    np.save(dir_final, pixels_n)
    
    return pixels_n


#6. ZERO CENTERING (¿queremos hacerlo?)--> https://www.kaggle.com/gzuidhof/full-preprocessing-tutorial
def zero_center(image, PIXEL_MEAN=0.25):
    image = image - PIXEL_MEAN
    return image


# VISUALIZAR CORTES
def sample_stack(stack, rows=5, cols=5, start_with=0, show_every=1):
    fig,ax = plt.subplots(rows,cols,figsize=[12,12])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title('slice %d' % ind)
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()
    
# VISUALIZAR CORTES
def hist(stack):
    fig= plt.hist(stack.flatten(), bins=80)
    plt.xlabel("Hounsfield Units (HU)")
    plt.ylabel("Frequency")
    plt.show()
    
#CREAR LOS CSV
def crear_csv(train_list, label_list, name_csv, data=[]):
    if name_csv is not 'TRAIN':
        train_list=np.transpose(train_list)
        label_list=np.transpose(label_list)

        data = {'name': train_list,
                'label': train_list}

    df = pd.DataFrame(data, columns = ['name', 'label']) 
    df.to_csv('/srv/data/HIC/datos/csv/DICOM'+name_csv+'.csv')    

In [25]:
###############################################################################################
#.........CREAMOS CSV Y HACEMOS EL PREPROCESADO DE LAS IMÁGENES PARA HIC Y CONTROL.............
############################################################################################### 
    
    
#CREAMOS EL EXCEL CON LOS PATHS Y LAS ETIQUETAS: 
#CASOS:
train_list = []
label_list = []

for dirpath, dirnames, fileList in os.walk('/srv/data/HIC/datos/HIC/DATASETS_HIC/'):
    for direc in dirnames:
        if direc is '.ipynb_checkpoints':
            print('es .ipynb_checkpoints')
        else:
            print('HIC_',direc)
            dir_final='/srv/data/HIC/datos/3d_dicom/HIC_'+str(direc)+'.npy'
            for dirpath1, dirnames1, fileList1 in os.walk('/srv/data/HIC/datos/HIC/DATASETS_HIC/'+str(direc)):

                slices = load_scan(dirpath1)
                print('[INFO] ANTES DE LA CONVERSIÓN')
                pixels = get_pixels_hu(slices)
                
                print('[INFO] DESPUÉS DE LA CONVERSIÓN')
                hist(np.array(pixels))
                pixels_n = preprocesado(pixels, slices, dir_final) 
                #pix_resampled, spacing = resample(pixels, slices, [5,5,5])
                
                print('[INFO] NORMALIZADO')
                hist(pixels_n)
                sample_stack(pixels_n, rows=4, cols=4, start_with=0, show_every=1)
                
                #resize dentro del pre procesado- no se puede usar cv2 entonces hay que buscar otra libreria            
                #lista para csv
                train_list.append(dir_final)
                label_list.append(1)
train_list_HIC=train_list
label_list_HIC=label_list     

#Crear CSV train
crear_csv(train_list, label_list, 'TRAIN_HIC')


#CONTROLES:
train_list = []
label_list = []

dirnames= !ls '/srv/data/HIC/datos/CONTROL/DICOM_NORMALES/CONTROLES_DICOM_CODIFICADOS/FILESET/'

for direc in dirnames:
    print('CONTROL_',direc)
    dir_final='/srv/data/HIC/datos/3d_dicom/CONTROL_'+str(direc)+'.npy'
    for dirpath1, dirnames1, fileList1 in os.walk('/srv/data/HIC/datos/CONTROL/DICOM_NORMALES/CONTROLES_DICOM_CODIFICADOS/FILESET/'+str(direc)+'/0/0/'):
        #print(dirpath1)
        #ORDENAR LOS CORTES
        slices = load_scan(dirpath1)
        print('[INFO] ANTES DE LA CONVERSIÓN')
        pixels = get_pixels_hu(slices)
                
        print('[INFO] DESPUÉS DE LA CONVERSIÓN')
        hist(np.array(pixels))
        
        pixels_n = preprocesado(pixels, slices, dir_final) 
        print('[INFO] NORMALIZADO')
        hist(pixels_n)
        sample_stack(pixels_n, rows=4, cols=4, start_with=0, show_every=1)
        
        #LISTA PARA HACER EL CSV
        train_list.append(dir_final)
        label_list.append(0)    

train_list_CONTROL=train_list
label_list_CONTROL=label_list
crear_csv(train_list, label_list, 'TRAIN_CONTROL')

train_list_CONTROL=np.transpose(train_list_CONTROL)
label_list_CONTROL=np.transpose(label_list_CONTROL)
train_list_HIC=np.transpose(train_list_HIC)
label_list_HIC=np.transpose(label_list_HIC)

names=np.concatenate((train_list_CONTROL, train_list_HIC))
labels=np.concatenate((label_list_CONTROL, label_list_HIC))

#TRAIN
data = {'name': names,'label': labels}
crear_csv(names, labels, '_', data)

HIC_ 99
[INFO] ANTES DE LA CONVERSIÓN


KeyboardInterrupt: 

In [ ]:

#VISUALIZACIÓN_FINAL_CASO(HIC):
imgs_to_process = np.load('/srv/data/HIC/datos/3d_dicom/CONTROL_47.npy', allow_pickle=True)
plt.hist(imgs_to_process.flatten(), bins=80)
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()
sample_stack(imgs_to_process, rows=4, cols=4, start_with=0, show_every=1)

#VISUALIZACIÓN_FINAL_CONTROL:
imgs_to_process = np.load('/srv/data/HIC/datos/3d_dicom/HIC_131.npy',allow_pickle=True)
plt.hist(imgs_to_process.flatten(), bins=80)
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()
sample_stack(imgs_to_process, rows=4, cols=4, start_with=0, show_every=1)

In [ ]:
test_split=0.20
SPLIT=5

In [ ]:
#CREAR KFOLDS

# read data
# Assuming it has two cols:
# name: path to each image with its extension
# label: labels (here it is 0s and 1s) -> binary classification
df = pd.read_csv('/srv/data/HIC/datos/csv/DICOM_.csv')
df = df.iloc[np.random.permutation(len(df))]
df = df.astype(str)

# split for testing
train_df, test_df = train_test_split(df, test_size=test_split)

print('[INFO] TRAIN CSV ',train_df.shape)
print('[INFO] TEST CSV ',test_df.shape)
train_df.to_csv('/srv/data/HIC/datos/csv/TRAIN.csv')
test_df.to_csv('/srv/data/HIC/datos/csv/TEST.csv')

from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit
    
Xtrain = train_df['name']
ytrain = train_df['label']

print("KFold, shuffle=False (default)")
kf = KFold(n_splits=SPLIT, random_state=True, shuffle=True)
i=1
X_train=[]
X_val=[]
y_train=[]
y_val=[]

for train_index, test_index in kf.split(Xtrain, ytrain):
    #print('TRAIN_'+str(i)+' :',train_index)
    #print('TEST_'+str(i)+' :', test_index)
    train_index=list(train_index)
    test_index=list(test_index)

    Xtrain = np.array(Xtrain)
    ytrain = np.array(ytrain)
    if(i is 1):
        X_train1 , X_val1 = Xtrain[train_index], Xtrain[test_index]
        y_train1, y_val1 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train1,
        'label': y_train1}
        dftrain1 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain1.to_csv('/srv/data/HIC/datos/csv/TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val1,
        'label': y_val1}
        dfval1 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval1.to_csv('/srv/data/HIC/datos/csv/VAL'+str(i)+'.csv')
        
        i=i+1
    elif(i is 2):
        X_train2 , X_val2 = Xtrain[train_index], Xtrain[test_index]
        y_train2, y_val2 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train2,
        'label': y_train2}
        dftrain2 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain2.to_csv('/srv/data/HIC/datos/csv/TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val2,
        'label': y_val2}
        dfval2 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval2.to_csv('/srv/data/HIC/datos/csv/VAL'+str(i)+'.csv')
        
        i=i+1
    elif(i is 3):
        X_train3 , X_val3 = Xtrain[train_index], Xtrain[test_index]
        y_train3, y_val3 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train3,
        'label': y_train3}
        dftrain3 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain3.to_csv('/srv/data/HIC/datos/csv/TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val3,
        'label': y_val3}
        dfval3 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval3.to_csv('/srv/data/HIC/datos/csv/VAL'+str(i)+'.csv')
        
        i=i+1
    elif(i is 4):
        X_train4 , X_val4 = Xtrain[train_index], Xtrain[test_index]
        y_train4, y_val4 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train4,
        'label': y_train4}
        dftrain4 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain4.to_csv('/srv/data/HIC/datos/csv/TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val4,
        'label': y_val4}
        dfval4 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval4.to_csv('/srv/data/HIC/datos/csv/VAL'+str(i)+'.csv')
        
        i=i+1
    elif(i is 5):
        X_train5 , X_val5 = Xtrain[train_index], Xtrain[test_index]
        y_train5, y_val5 = ytrain[train_index], ytrain[test_index]
        
        data = {'name': X_train5,
        'label': y_train5}
        dftrain5 = pd.DataFrame(data, columns = ['name', 'label']) 
        dftrain5.to_csv('/srv/data/HIC/datos/csv/TRAIN'+str(i)+'.csv')
        
        data = {'name': X_val5,
        'label': y_val5}
        dfval5 = pd.DataFrame(data, columns = ['name', 'label']) 
        dfval5.to_csv('/srv/data/HIC/datos/csv/VAL'+str(i)+'.csv')       
        i=i+1
        
#REVISAR KFOLDS HECHOS Y TAMAÑO   
print('\nTRAIN 1',dftrain1.shape)
print('VAL 1', dfval1.shape)

print('TRAIN 2',dftrain2.shape)
print('VAL 2',dfval2.shape)

print('TRAIN 3',dftrain3.shape)
print('VAL 3',dfval3.shape)

print('TRAIN 4',dftrain4.shape)
print('VAL 4',dfval4.shape)

print('TRAIN 5',dftrain5.shape)
print('VAL 5',dfval5.shape)